In [1]:
# Import Regex
import re
import pandas as pd

In [2]:
# Identify different formats of phone number (US)
def is_phone_number(phone):
    patterns = [
        r'^\d{3}-\d{3}-\d{4}$',            # 123-456-7890
        r'^\(\d{3}\)\s\d{3}-\d{4}$',       # (123) 456-7890
        r'^\d{3}\.\d{3}\.\d{4}$',          # 123.456.7890
        r'^\+?\d{1,2}-\d{3}-\d{3}-\d{4}$', # +1-123-456-7890 or 1-123-456-7890
        r'^\d\s?\(\d\s?\d\s?\d\)\s?\d\s?\d\s?\d\sdash\s?\d\s?\d\s?\d\s?\d$',          # 1 (123) 456-7890
        r'^\d{10}$',                       # 1234567890
        r'^\d{3}\s\d{3}\s\d{4}$',          # 123 456 7890
        r'^(?:\d\s?){10}$',                # 1 2 3 4 5 6 7 8 9 0
        r'^\(\d{3}\)\s?-\s?\d{3}-\d{4}$',  # (111) - 222 - 3333 (allowing optional spaces around dashes)

        # To match stranger formats
        # r'^\+?\d{1,3}[\s.-]\d{3}[\s.-]\d{3}[\s.-]\d{4}$',  # +553 575-381.4440, +21.13 926 7829
        # r'^00\d{1,3}[\s.-]?\d{2,3}[\s.-]?\d{3}[\s.-]?\d{4}$',  # 002716236.1690, 0049 48 490.5598
    ]
    area_codes = [
        '205', '251', '256', '334', '659', '938',  # Alabama
        '907',  # Alaska
        '480', '520', '602', '623', '928',  # Arizona
        '327', '479', '501', '870',  # Arkansas
        '209', '213', '279', '310', '323', '341', '350', '369', '408', '415', '424', '442', '510', '530', '559', '562', '619', '626', '628', '650', '657', '661', '669', '707', '714', '747', '760', '805', '818', '820', '831', '840', '858', '909', '916', '925', '949', '951',  # California
        '303', '719', '720', '970', '983',  # Colorado
        '203', '475', '860', '959',  # Connecticut
        '302',  # Delaware
        '239', '305', '321', '324', '352', '386', '407', '448', '561', '645', '656', '689', '727', '728', '754', '772', '786', '813', '850', '863', '904', '941', '954',  # Florida
        '229', '404', '470', '478', '678', '706', '762', '770', '912', '943',  # Georgia
        '808',  # Hawaii
        '208', '986',  # Idaho
        '217', '224', '309', '312', '331', '447', '464', '618', '630', '708', '730', '773', '779', '815', '847', '861', '872',  # Illinois
        '219', '260', '317', '463', '574', '765', '812', '930',  # Indiana
        '319', '515', '563', '641', '712',  # Iowa
        '316', '620', '785', '913',  # Kansas
        '270', '364', '502', '606', '859',  # Kentucky
        '225', '318', '337', '504', '985',  # Louisiana
        '207',  # Maine
        '227', '240', '301', '410', '443', '667',  # Maryland
        '339', '351', '413', '508', '617', '774', '781', '857', '978',  # Massachusetts
        '231', '248', '269', '313', '517', '586', '616', '734', '810', '906', '947', '989',  # Michigan
        '218', '320', '507', '612', '651', '763', '952',  # Minnesota
        '228', '601', '662', '769',  # Mississippi
        '235', '314', '417', '557', '573', '636', '660', '816', '975',  # Missouri
        '406',  # Montana
        '308', '402', '531',  # Nebraska
        '702', '725', '775',  # Nevada
        '603',  # New Hampshire
        '201', '551', '609', '640', '732', '848', '856', '862', '908', '973',  # New Jersey
        '505', '575',  # New Mexico
        '212', '315', '329', '332', '347', '363', '516', '518', '585', '607', '624', '631', '646', '680', '716', '718', '838', '845', '914', '917', '929', '934',  # New York
        '252', '336', '472', '704', '743', '828', '910', '919', '980', '984',  # North Carolina
        '701',  # North Dakota
        '216', '220', '234', '283', '326', '330', '380', '419', '436', '440', '513', '567', '614', '740', '937',  # Ohio
        '405', '539', '572', '580', '918',  # Oklahoma
        '458', '503', '541', '971',  # Oregon
        '215', '223', '267', '272', '412', '445', '484', '570', '582', '610', '717', '724', '814', '835', '878',  # Pennsylvania
        '401',  # Rhode Island
        '803', '839', '843', '854', '864',  # South Carolina
        '605',  # South Dakota
        '423', '615', '629', '731', '865', '901', '931',  # Tennessee
        '210', '214', '254', '281', '325', '346', '361', '409', '430', '432', '469', '512', '682', '713', '726', '737', '806', '817', '830', '832', '903', '915', '936', '940', '945', '956', '972', '979',  # Texas
        '385', '435', '801',  # Utah
        '802',  # Vermont
        '276', '434', '540', '571', '686', '703', '757', '804', '826', '948',  # Virginia
        '206', '253', '360', '425', '509', '564',  # Washington
        '202', '771',  # Washington, DC
        '304', '681',  # West Virginia
        '262', '274', '353', '414', '534', '608', '715', '920',  # Wisconsin
        '307',  # Wyoming
        '684',  # American Samoa
        '671',  # Guam
        '670',  # Northern Mariana Islands
        '787', '939',  # Puerto Rico
        '340'  # Virgin Islands
    ]

    # # Check if the phone number matches any of the patterns
    # # Does not include area code
    # for pattern in patterns:
    #     if re.match(pattern, phone):
    #         # Check that area code is valid
    #         match = re.search(r'\(?(\d{3})\)?[-\s.]?\d{3}[-\s.]?\d{4}', phone)
    #         if match:
    #             return True
    # return False

    # Include area code in the criteria
    for pattern in patterns:
        if re.match(pattern, phone):
            # Extract the area code from the phone number
            match = re.search(r'\(?(\d{3})\)?[-\s.]?\d{3}[-\s.]?\d{4}', phone)
            if match:
                area_code = match.group(1)  # Extract the area code
                if area_code in area_codes:
                    return True  # The phone number is valid and has a valid area code
                else:
                    return False  # Invalid area code
    return False

In [3]:
# Test 1
test_inputs = [
    # should evaluate to True
    '(123) 456-7890',
    '123-456-7890',
    '482.950.3182',
    '+1-123-456-7890',
    '3489820455',
    '(669)-246-2485',

    # should evaluate to False
    '(111) - 222 - 3333',
    # '+553 575-381.4440', 
    # '002716236.1690', 
    # '0049 48 490.5598', 
    '+21.13 926 7829', # invalid
    '+41 15-968.9292', # invalid
    '1234-567-890'     # invalid
]

for text in test_inputs:
    print(f'{text}: {is_phone_number(text)}')

(123) 456-7890: False
123-456-7890: False
482.950.3182: False
+1-123-456-7890: False
3489820455: False
(669)-246-2485: True
(111) - 222 - 3333: False
+21.13 926 7829: False
+41 15-968.9292: False
1234-567-890: False


In [4]:
# Test the algorithm using the dataset
filename = 'extracted_tel_data.csv'
df = pd.read_csv(filename)

In [5]:
# Regex pattern to extract potential phone numbers from text
phone_pattern = re.compile(r'[\d\s\-\.\(\)]{7,}')

In [6]:
# Function to extract and validate phone numbers
def extract_validate_phone(text):
    potential_phone = phone_pattern.findall(text)
    valid_phone = [num.strip() for num in potential_phone if is_phone_number(num.strip())]
    return valid_phone

In [7]:
# Apply the function to each row in the df
df['valid_phone'] = df['unmasked_text'].apply(extract_validate_phone)

# Display rows where valid phone numbers were found
valid_phone_df = df[df['valid_phone'].apply(len) > 0]

valid_phone_df

,unmasked_text,masked_text,valid_phone
975,"Recently, an abnormal transaction was observed...","Recently, an abnormal transaction was observed...",[8596250047]
3037,Assetbewertung benötigt für Will - Stehr. Kont...,Assetbewertung benötigt für [COMPANYNAME_1]. K...,[52-206-857-8850]
3243,Terminvereinbarung für eine private Sitzung mi...,Terminvereinbarung für eine private Sitzung mi...,[28-656-376-7767]
5445,"Bonjour Kennedy, je vous contacte au nom de la...","Bonjour [MIDDLENAME_1], je vous contacte au no...",[41-804-169-3942]
5948,"Hallo Leslie, wenn Cooper Unterstützung bei de...","Hallo [MIDDLENAME_1], wenn [FIRSTNAME_1] Unter...",[96-728-553-2446]
6338,Our online Positive Psychology course has a sp...,Our online Positive Psychology course has a sp...,[2077653891]
6535,Die Schulbibliothek in Kossshire ist am 24/11/...,Die Schulbibliothek in [CITY_1] ist am [DATE_1...,[88-313-551-8947]
6917,"Guten Tag Mr. Baumbach, ich kontaktiere Sie be...","Guten Tag [PREFIX_1] [LASTNAME_1], ich kontakt...",[63-424-194-3390]
8927,"Dear Miss Cruickshank, as we are going digital...","Dear [PREFIX_1] [LASTNAME_1], as we are going ...",[13-447-488-6146]
9572,"Mrs. Huel, solo un promemoria riguardante la r...","[PREFIX_1] [LASTNAME_1], solo un promemoria ri...",[88-903-830-5431]
